In [1]:
import pickle
import base64
import os
import json
import requests

In [2]:
globus_env_data = os.getenv('GLOBUS_DATA')

In [3]:
print(globus_env_data)

gAN9cQAoWAkAAABjbGllbnRfaWRxAVgkAAAANGI3MjUyNDUtOTU3Ni00ODhhLThkY2ItNDdkZDY4MWFlZDE4cQJYBgAAAHRva2Vuc3EDfXEEKFgPAAAAYXV0aC5nbG9idXMub3JncQV9cQYoWAUAAABzY29wZXEHWA4AAABwcm9maWxlIG9wZW5pZHEIWAwAAABhY2Nlc3NfdG9rZW5xCVhfAAAAQWdvNVBNUE83YTIwN1FuUG9ORWRZRzB5NjU5RHk0bjBnd0puOHFWZWpXYWR4eGQ2MnZpZ0NKMVFkSm9yTTFsemo5SjFRN1Btb0JYQmVESUs5cmtLRVQ4a0VtY2tsYW9xClgNAAAAcmVmcmVzaF90b2tlbnELTlgKAAAAdG9rZW5fdHlwZXEMWAYAAABCZWFyZXJxDVgSAAAAZXhwaXJlc19hdF9zZWNvbmRzcQ5KrFDeWnVYFwAAAHRyYW5zZmVyLmFwaS5nbG9idXMub3JncQ99cRAoaAdYMQAAAHVybjpnbG9idXM6YXV0aDpzY29wZTp0cmFuc2Zlci5hcGkuZ2xvYnVzLm9yZzphbGxxEWgJWFoAAABBZzFnZWUzZXJFcjY3Qlk3YnZxbW5lOG9kNmwzVmQzNXkwRE5uYTZwb3g0RWI0a3l2ZXUyQ2oyZ0I0N3hnUUJ5eTRWMVgxWHpLeERyT2toNDk4WDRlc2dxRDVxEmgLTmgMWAYAAABCZWFyZXJxE2gOSqxQ3lp1dVgIAAAAaWRfdG9rZW5xFH1xFShYHgAAAGlkZW50aXR5X3Byb3ZpZGVyX2Rpc3BsYXlfbmFtZXEWWAkAAABHbG9idXMgSURxF1gDAAAAc3VicRhYJAAAAGM0ZTE2ZTYyLWQyNzQtMTFlNS1iOWU5LWNmMDIyNTA2OGNlOHEZWAMAAABpc3NxGlgXAAAAaHR0cHM6Ly9hdXRoLmdsb2J1cy5vcmdxG1gSAAAAcHJlZmVycmVkX3VzZXJuYW1l

In [4]:
pickled_tokens = base64.b64decode(globus_env_data)

In [5]:
tokens = pickle.loads(pickled_tokens)

In [6]:
print(json.dumps(tokens, indent=4, sort_keys=True))

{
    "client_id": "4b725245-9576-488a-8dcb-47dd681aed18",
    "id_token": {
        "at_hash": "GXnXh6k1vRengGqJTHOy9IRow2o_zAQ0QqsI3KBaZxA",
        "aud": "4b725245-9576-488a-8dcb-47dd681aed18",
        "exp": 1524519084,
        "iat": 1524346284,
        "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
        "identity_provider_display_name": "Globus ID",
        "iss": "https://auth.globus.org",
        "name": "Rick Wagner",
        "organization": "Globus",
        "preferred_username": "rpwagner@globusid.org",
        "sub": "c4e16e62-d274-11e5-b9e9-cf0225068ce8"
    },
    "tokens": {
        "auth.globus.org": {
            "access_token": "Ago5PMPO7a207QnPoNEdYG0y659Dy4n0gwJn8qVejWadxxd62vigCJ1QdJorM1lzj9J1Q7PmoBXBeDIK9rkKET8kEmcklao",
            "expires_at_seconds": 1524519084,
            "refresh_token": null,
            "scope": "profile openid",
            "token_type": "Bearer"
        },
        "transfer.api.globus.org": {
            "access_token"

In [10]:
headers = {'Authorization':'Bearer '+ tokens['tokens']['auth.globus.org']['access_token']}
user_info = requests.get('https://auth.globus.org/v2/oauth2/userinfo',headers=headers).json()
print(json.dumps(user_info, indent=4, sort_keys=True))

{
    "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
    "identity_provider_display_name": "Globus ID",
    "name": "Rick Wagner",
    "organization": "Globus",
    "preferred_username": "rpwagner@globusid.org",
    "sub": "c4e16e62-d274-11e5-b9e9-cf0225068ce8"
}


In [11]:
identity = requests.get('https://auth.globus.org/v2/api/identities/' + user_info['sub'],headers=headers).json()
print(json.dumps(identity, indent=4, sort_keys=True))

{
    "identity": {
        "email": "rick@globus.org",
        "id": "c4e16e62-d274-11e5-b9e9-cf0225068ce8",
        "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
        "name": "Rick Wagner",
        "organization": "Globus",
        "status": "used",
        "username": "rpwagner@globusid.org"
    }
}


In [12]:
# http://statweb.stanford.edu/~sabatti/data.html
net_data = requests.get('http://hci.stanford.edu/courses/cs448b/data/ipasn/cs448b_ipasn.csv').text